In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

In [20]:
# Data augmentation for training
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2,
    saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225])
])

# No augmentation for validation/test
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225])
])

In [21]:
# Load datasets
train_dataset = datasets.ImageFolder('data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('data/val', transform=val_transforms)
test_dataset = datasets.ImageFolder('data/test', transform=val_transforms)

In [22]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')

Training samples: 19999
Validation samples: 2500
Test samples: 2460


In [25]:
# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [28]:
def train_and_evaluate(model_name):
    print(f"\n{'='*20} Training {model_name} {'='*20}")

    if model_name == 'ResNet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False

        # Replace final layer for binary classification
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)
        optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

    elif model_name == 'MobileNetV2':
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False

        # Replace final layer for binary classification
        # MobileNetV2 classifier is a Sequential block, last layer is Linear
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, 2)
        optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)

    else:
        raise ValueError(f"Unknown model name: {model_name}")

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

    best_val_acc = 0.0
    best_model_path = f'best_model_{model_name}.pth'

    num_epochs = 10 # Keep same as original

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        # Update learning rate
        scheduler.step(val_loss)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with val_acc: {val_acc:.2f}%')

        print(f'Epoch [{epoch+1}/{num_epochs}] '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    # Load best model for testing
    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = 100 * test_correct / test_total
    print(f'{model_name} Test Accuracy: {test_accuracy:.2f}%')

    # return test_accuracy, train_losses, val_losses, train_accs, val_accs
    return test_accuracy

In [29]:
# Compare Models
resnet_acc = train_and_evaluate('ResNet18')
mobile_acc = train_and_evaluate('MobileNetV2')

print(f"\n{'='*20} Comparison Results {'='*20}")
print(f"ResNet18 Accuracy: {resnet_acc:.2f}%")
print(f"MobileNetV2 Accuracy: {mobile_acc:.2f}%")

if resnet_acc > mobile_acc:
    print(f"Result: ResNet18 performs better by {resnet_acc - mobile_acc:.2f}%")
elif mobile_acc > resnet_acc:
    print(f"Result: MobileNetV2 performs better by {mobile_acc - resnet_acc:.2f}%")
else:
    print("Result: Both models performed equally.")


==================== Training ResNet18 ====================


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Saved best model with val_acc: 97.80%
Epoch [1/10] Train Loss: 0.1640, Train Acc: 93.41% | Val Loss: 0.0612, Val Acc: 97.80%
Saved best model with val_acc: 98.04%
Epoch [2/10] Train Loss: 0.1142, Train Acc: 95.29% | Val Loss: 0.0531, Val Acc: 98.04%
Saved best model with val_acc: 98.20%
Epoch [3/10] Train Loss: 0.1111, Train Acc: 95.53% | Val Loss: 0.0518, Val Acc: 98.20%
Epoch [4/10] Train Loss: 0.1121, Train Acc: 95.47% | Val Loss: 0.0513, Val Acc: 98.20%
Epoch [5/10] Train Loss: 0.1104, Train Acc: 95.45% | Val Loss: 0.0704, Val Acc: 97.40%
Saved best model with val_acc: 98.24%
Epoch [6/10] Train Loss: 0.1097, Train Acc: 95.49% | Val Loss: 0.0478, Val Acc: 98.24%
Epoch [7/10] Train Loss: 0.1070, Train Acc: 95.65% | Val Loss: 0.0547, Val Acc: 97.80%
Epoch [8/10] Train Loss: 0.1072, Train Acc: 95.68% | Val Loss: 0.0506, Val Acc: 98.08%
Epoch [9/10] Train Loss: 0.1027, Train Acc: 95.86% | Val Loss: 0.0510, Val Acc: 98.12%
Epoch [10/10] Train Loss: 0.0997, Train Acc: 96.08% | Val Loss: 0

100%|██████████| 13.6M/13.6M [00:00<00:00, 167MB/s]


Saved best model with val_acc: 97.20%
Epoch [1/10] Train Loss: 0.1488, Train Acc: 93.92% | Val Loss: 0.0666, Val Acc: 97.20%
Saved best model with val_acc: 97.72%
Epoch [2/10] Train Loss: 0.1310, Train Acc: 94.69% | Val Loss: 0.0642, Val Acc: 97.72%
Epoch [3/10] Train Loss: 0.1267, Train Acc: 94.89% | Val Loss: 0.0700, Val Acc: 97.12%
Epoch [4/10] Train Loss: 0.1219, Train Acc: 95.05% | Val Loss: 0.0623, Val Acc: 97.52%
Epoch [5/10] Train Loss: 0.1324, Train Acc: 94.76% | Val Loss: 0.0609, Val Acc: 97.68%
Epoch [6/10] Train Loss: 0.1293, Train Acc: 94.88% | Val Loss: 0.0689, Val Acc: 97.36%
Epoch [7/10] Train Loss: 0.1217, Train Acc: 95.15% | Val Loss: 0.0920, Val Acc: 96.60%
Saved best model with val_acc: 97.80%
Epoch [8/10] Train Loss: 0.1231, Train Acc: 95.22% | Val Loss: 0.0577, Val Acc: 97.80%
Epoch [9/10] Train Loss: 0.1328, Train Acc: 94.77% | Val Loss: 0.0692, Val Acc: 97.00%
Epoch [10/10] Train Loss: 0.1252, Train Acc: 95.03% | Val Loss: 0.0618, Val Acc: 97.72%
MobileNetV2 Tes